# Task #1

## 1. Algoritmo Minimax en Juegos de Suma Cero

El algoritmo Minimax es un método de toma de decisiones utilizado en juegos de dos jugadores de suma cero con información perfecta (como el ajedrez o el to-ti-to). Su objetivo es encontrar la estrategia óptima para cada jugador suponiendo que ambos juegan de manera óptima.

### ¿Cómo funciona?

* Se representa el juego como un árbol de decisiones, donde los nodos representan estados del juego y las ramas representan movimientos posibles.
* Se asigna un valor a cada estado terminal (hoja), dependiendo de quién gana o pierde.
* El jugador MAX busca maximizar su ganancia, eligiendo el movimiento con el valor más alto.
* El jugador MIN (el oponente) busca minimizar la ganancia de MAX, eligiendo el movimiento con el valor más bajo.
* El valor minimax de un nodo es el valor que un jugador puede garantizarse si ambos juegan de manera óptima.
* El proceso se aplica recursivamente hasta llegar a la raíz del árbol, determinando el mejor movimiento inicial.

### Importancia del Valor Minimax

El valor minimax representa el mejor resultado garantizado para un jugador bajo el supuesto de que el oponente también juega óptimamente. Es clave en la toma de decisiones estratégicas en juegos competitivos.

## 2. Minimax vs. Poda Alfa-Beta

La poda alfa-beta es una optimización del algoritmo minimax que reduce la cantidad de nodos evaluados, mejorando la eficiencia sin afectar el resultado final.

### Diferencia Principal

Minimax examina todos los nodos del árbol de juego, mientras que la poda alfa-beta descarta ramas innecesarias, reduciendo el tiempo de ejecución.

### ¿Cómo funciona la poda alfa-beta?

* Usa dos valores:
    * Alfa (α): el mejor valor que MAX puede garantizar.
    * Beta (β): el mejor valor que MIN puede garantizar.
* Si en un nodo MIN se encuentra un valor menor que α, no se sigue explorando esa rama (porque MAX nunca la elegirá).
* Si en un nodo MAX se encuentra un valor mayor que β, se deja de explorar la rama (porque MIN nunca permitirá que se llegue ahí).

### Ejemplo de Complejidad

Supongamos un árbol de profundidad d y con b opciones por nivel:

* Minimax: O(b^d) (explora todos los nodos).
* Poda Alfa-Beta (con orden óptimo): O(b^(d/2)) (se exploran muchos menos nodos).

Esto significa que la poda alfa-beta reduce a la mitad la profundidad efectiva del árbol en el mejor caso.

## 3. Expectiminimax en Juegos con Incertidumbre

El expectiminimax es una extensión del algoritmo minimax para juegos donde hay elementos de azar, como los dados en el juego de Backgammon.

### Diferencias con Minimax

* Minimax asume que los jugadores toman decisiones deterministas y estratégicas.
* Expectiminimax incorpora nodos de azar que representan eventos probabilísticos, como tirar un dado o robar una carta.

### ¿Cómo funciona?

* Al igual que minimax, hay nodos MAX y MIN que representan jugadores.
* Se agregan nodos de azar donde se calculan valores esperados ponderados según las probabilidades de cada evento.
* En lugar de seleccionar el mejor o peor resultado, se toma la esperanza matemática de los posibles valores.

### Desafíos Claves

* Complejidad computacional: Al agregar nodos de azar, el árbol crece mucho más rápido.
* Estimación de probabilidades: Se requieren distribuciones de probabilidad precisas para hacer buenas decisiones.
* Juegos con información oculta: Como el póker, donde no conocemos todas las cartas del oponente, lo que hace más difícil calcular expectativas precisas.